## Imports

In [41]:
import os
import numpy as np
from numpy.random import default_rng
import cv2
import imageio
import imgaug as ia
import imgaug.augmenters as iaa

## Settings

In [42]:
dataset = '/home/drevital/obstacles_classification_datasets/rgb_6_C/enhanced'
obstacle_dataset = os.path.join(dataset, 'obstacle')
no_obstacle_dataset = os.path.join(dataset, 'no_obstacle')
aug_dataset = '/home/drevital/obstacles_classification_datasets/rgb_6_C/augmented'
obstacle_augment_dataset = os.path.join(aug_dataset, 'obstacle')
no_obstacle_augment_dataset = os.path.join(aug_dataset, 'no_obstacle')
obstacle_fnames = os.listdir(obstacle_dataset)
no_obstacle_fnames = os.listdir(no_obstacle_dataset)
current_obs = len(obstacle_fnames)
current_no_obs = len(no_obstacle_fnames)
target = 10000
augments_obs = target - current_obs
augments_no_obs = target - current_no_obs

In [43]:
current_obs, current_no_obs, target, augments_obs, augments_no_obs

(6703, 6235, 10000, 3297, 3765)

## Augmentation Functions

In [44]:
def rotate(i, im, im1, im2):
    degrees = np.random.randint(-360, 361, size=augments)
    rotate = iaa.Affine(rotate=(degrees[i], degrees[i]))
    rotated_im1 = rotate.augment_image(im1)
    rotated_im2 = rotate.augment_image(im2)
    rotated_image = cv2.hconcat([rotated_im1, rotated_im2])

    return rotated_image

def noise(i, im, im1, im2):
    gaussian_noise = iaa.AdditiveGaussianNoise(10, 20)
    noise_image = gaussian_noise.augment_image(im)
        
    return noise_image

def crop(i, im, im1, im2):
    percents = np.random.randint(0, 5, size=augments)
    percent_ind = i % len(percents)
    percent = percents[percent_ind] / 100.0
    crop = iaa.Crop(percent=(percent, percent))
    crop_im1 = crop.augment_image(im1)
    crop_im2 = crop.augment_image(im2)
    crop_image = cv2.hconcat([crop_im1, crop_im2])

    return crop_image

def shear(i, im, im1, im2):
    shears = np.random.randint(0, 41, size=augments)
    shear_ind = i % len(shears)
    shear = iaa.Affine(shear=(shears[shear_ind], shears[i]))
    shear_im1 = shear.augment_image(im1)
    shear_im2 = shear.augment_image(im2)
    shear_image = cv2.hconcat([shear_im1, shear_im2])

    return shear_image

def contrast(i, im, im1, im2):
    gammas = np.random.randint(70, 171, size=augments)
    gamma = gammas / 100.0
    contrast = iaa.GammaContrast(gamma=gamma)
    contrast_im1 = contrast.augment_image(im1)
    contrast_im2 = contrast.augment_image(im2)
    contrast_image = cv2.hconcat([contrast_im1, contrast_im2])

    return contrast_image

def vertical_flip(i, im, im1, im2):
    flip_vr = iaa.Flipud(p=1.0)
    flip_vr_image = flip_vr.augment_image(im)

    return flip_vr_image

## Augmentation Handling Function

In [45]:
def augment(dataset,
            aug_dataset,
            aug_dir,
            fnames,
            num_augments,
            rands,
            aug_names,
            display=False):
    
    for i, rand in enumerate(rands):    
        in_fname = fnames[rand]
        in_imname = in_fname.split('.')[0]
        in_impath = os.path.join(dataset, aug_dir, in_fname)
        im = imageio.imread(in_impath)
        w = im.shape[1]
        im1 = im[:, :w//2]
        im2 = im[:, w//2:]
        naugment = i % num_augments
        augmented = augment_fn[naugment](i, im, im1, im2)
        if display:
            ia.imshow(augmented)
        out_imname = in_imname + aug_names[naugment] + str(i)
        out_impath = os.path.join(aug_dataset, aug_dir, out_imname + '.jpg')
        imageio.imwrite(out_impath, augmented)

## Augment

In [46]:
rng = default_rng()
rand_obs = rng.choice(current_obs, size=augments_obs, replace=True)
rand_no_obs = rng.choice(current_no_obs, size=augments_no_obs, replace=True)
augment_fn = [rotate, noise, crop, shear, contrast, vertical_flip]
num_augments = len(augment_fn)
aug_names = ['_rotate_',
                '_noise_',
                '_crop_',
                '_shear_',
                '_contrast_',
                '_vr_flip_']

augment(dataset,
        aug_dataset,
        'obstacle',
        obstacle_fnames,
        num_augments,
        rand_obs,
        aug_names)

augment(dataset,
        aug_dataset,
        'no_obstacle',
        no_obstacle_fnames,
        num_augments,
        rand_no_obs,
        aug_names)